# EKSクラスタを作成

## 新規作成する場合

In [2]:
clusterName = 'aws-eks-200127'

In [3]:
nodeGroupName = 'ng-200127'

In [4]:
!eksctl create cluster \
    --vpc-cidr 10.0.0.0/16 \
    --vpc-nat-mode HighlyAvailable \
    --name {clusterName} \
    --version 1.14 \
    --nodegroup-name {nodeGroupName} \
    --node-type t3.small \
    --nodes 2 \
    --zones=us-west-2b,us-west-2c \
    --managed

[ℹ]  eksctl version 0.13.0
[ℹ]  using region us-west-2
[ℹ]  subnets for us-west-2b - public:10.0.0.0/19 private:10.0.64.0/19
[ℹ]  subnets for us-west-2c - public:10.0.32.0/19 private:10.0.96.0/19
[ℹ]  using Kubernetes version 1.14
[ℹ]  creating EKS cluster "aws-eks-200127" in "us-west-2" region with managed nodes
[ℹ]  will create 2 separate CloudFormation stacks for cluster itself and the initial managed nodegroup
[ℹ]  if you encounter any issues, check CloudFormation console or try 'eksctl utils describe-stacks --region=us-west-2 --cluster=aws-eks-200127'
[ℹ]  CloudWatch logging will not be enabled for cluster "aws-eks-200127" in "us-west-2"
[ℹ]  you can enable it with 'eksctl utils update-cluster-logging --region=us-west-2 --cluster=aws-eks-200127'
[ℹ]  Kubernetes API endpoint access will use default of {publicAccess=true, privateAccess=false} for cluster "aws-eks-200127" in "us-west-2"
[ℹ]  2 sequential tasks: { create cluster control plane "aws-eks-200127", create managed nodegroup

クラスターができていることを確認する

In [11]:
!kubectl get svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   172.20.0.1   <none>        443/TCP   16m


# ALBの構築

IngressからALBを作成するには、IAMロールやら、AWS ALB Ingress Controllerやらの設定が必要。


https://docs.aws.amazon.com/ja_jp/eks/latest/userguide/alb-ingress.html

ロードバランサーに使用する VPC のサブネットにタグを付け、そのサブネットを使用できることを ALB Ingress Controller に伝えます。詳細については、「サブネットのタグ付け要件」を参照してください。

PublicとPrivateを別々に取得

In [15]:
row = !aws ec2 describe-subnets --filters "Name=tag:Name,Values=*{clusterName}*Public*"

import json
import pprint

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
publicSubnets = json.loads(''.join(row))
pprint.pprint(publicSubnets)

{'Subnets': [{'AssignIpv6AddressOnCreation': False,
              'AvailabilityZone': 'us-west-2b',
              'AvailabilityZoneId': 'usw2-az2',
              'AvailableIpAddressCount': 8182,
              'CidrBlock': '10.0.0.0/19',
              'DefaultForAz': False,
              'Ipv6CidrBlockAssociationSet': [],
              'MapPublicIpOnLaunch': False,
              'OwnerId': '565991835156',
              'State': 'available',
              'SubnetArn': 'arn:aws:ec2:us-west-2:565991835156:subnet/subnet-0f973f5322f5ac12a',
              'SubnetId': 'subnet-0f973f5322f5ac12a',
              'Tags': [{'Key': 'aws:cloudformation:stack-name',
                        'Value': 'eksctl-aws-eks-200127-cluster'},
                       {'Key': 'alpha.eksctl.io/cluster-name',
                        'Value': 'aws-eks-200127'},
                       {'Key': 'eksctl.cluster.k8s.io/v1alpha1/cluster-name',
                        'Value': 'aws-eks-200127'},
                       {'Key'

In [16]:
row = !aws ec2 describe-subnets --filters "Name=tag:Name,Values=*{clusterName}*Private*"

import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
privateSubnets = json.loads(''.join(row))
pprint.pprint(privateSubnets)

{'Subnets': [{'AssignIpv6AddressOnCreation': False,
              'AvailabilityZone': 'us-west-2b',
              'AvailabilityZoneId': 'usw2-az2',
              'AvailableIpAddressCount': 8186,
              'CidrBlock': '10.0.64.0/19',
              'DefaultForAz': False,
              'Ipv6CidrBlockAssociationSet': [],
              'MapPublicIpOnLaunch': False,
              'OwnerId': '565991835156',
              'State': 'available',
              'SubnetArn': 'arn:aws:ec2:us-west-2:565991835156:subnet/subnet-0914edd00a3a9a9e5',
              'SubnetId': 'subnet-0914edd00a3a9a9e5',
              'Tags': [{'Key': 'aws:cloudformation:stack-id',
                        'Value': 'arn:aws:cloudformation:us-west-2:565991835156:stack/eksctl-aws-eks-200127-cluster/e27c42b0-40a4-11ea-9eba-0a90939a96ce'},
                       {'Key': 'eksctl.cluster.k8s.io/v1alpha1/cluster-name',
                        'Value': 'aws-eks-200127'},
                       {'Key': 'kubernetes.io/cluster/aw

Kubernetes 用のタグはすでに付与されていることを確認した。

ワーカーノードインスタンスプロファイルに対して、ALBIngressControllerIAMPolicy という IAM ポリシーを作成し、ALB Ingress Controller がユーザーに代わって AWS API を呼び出せるようにします。次の AWS CLI コマンドを使用して、AWS アカウントで IAM ポリシーを作成します。GitHub でポリシードキュメントを表示することもできます。

GitHub からポリシードキュメントをダウンロードします。

In [85]:
!wget -O temp/iam-policy.json https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/iam-policy.json

--2020-01-27 02:35:59--  https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/iam-policy.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3546 (3.5K) [text/plain]
Saving to: ‘temp/iam-policy.json’

iam-policy.json     100%[===================>]   3.46K  --.-KB/s    in 0s      

2020-01-27 02:35:59 (63.7 MB/s) - ‘temp/iam-policy.json’ saved [3546/3546]



In [86]:
!cat temp/iam-policy.json

{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "acm:DescribeCertificate",
        "acm:ListCertificates",
        "acm:GetCertificate"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:AuthorizeSecurityGroupIngress",
        "ec2:CreateSecurityGroup",
        "ec2:CreateTags",
        "ec2:DeleteTags",
        "ec2:DeleteSecurityGroup",
        "ec2:DescribeAccountAttributes",
        "ec2:DescribeAddresses",
        "ec2:DescribeInstances",
        "ec2:DescribeInstanceStatus",
        "ec2:DescribeInternetGateways",
        "ec2:DescribeNetworkInterfaces",
        "ec2:DescribeSecurityGroups",
        "ec2:DescribeSubnets",
        "ec2:DescribeTags",
        "ec2:DescribeVpcs",
        "ec2:ModifyInstanceAttribute",
        "ec2:ModifyNetworkInterfaceAttribute",
        "ec2:RevokeSecurityGroupIngress"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "A

ポリシーを作成します。

In [89]:
row = !aws iam create-policy \
--policy-name ALBIngressControllerIAMPolicy \
--policy-document file://temp/iam-policy.json

import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
policy = json.loads(''.join(row))
pprint.pprint(policy)

{'Policy': {'Arn': 'arn:aws:iam::565991835156:policy/ALBIngressControllerIAMPolicy',
            'AttachmentCount': 0,
            'CreateDate': '2020-01-27T02:37:44Z',
            'DefaultVersionId': 'v1',
            'IsAttachable': True,
            'Path': '/',
            'PermissionsBoundaryUsageCount': 0,
            'PolicyId': 'ANPAYHR56AYKJS4GSNBZ3',
            'PolicyName': 'ALBIngressControllerIAMPolicy',
            'UpdateDate': '2020-01-27T02:37:44Z'}}


In [20]:
ALBPolicyArn = policy['Policy']['Arn']
print(ALBPolicyArn)

arn:aws:iam::565991835156:policy/ALBIngressControllerIAMPolicy


ワーカーノード用の IAM ロール名を取得します。次のコマンドを使用して、aws-auth configmap を出力します。

In [21]:
row = !kubectl -n kube-system describe configmap aws-auth

system:nodes グループが割り当てられている rolearn 値のロール名を取り出します。クラスターの各ノードグループに 1 つの値が必要です。

In [22]:
import re

m = re.search(r'arn:aws:iam::([0-9]+):role/(.*)', row[12])

pprint.pprint(m.group(2))

workerRolearn = m.group(2)


'eksctl-aws-eks-200127-nodegroup-n-NodeInstanceRole-B5AZPMU1PGAA'


次のコマンドを使用して、前に識別した各ワーカーノードの IAM ロールに、新しい ALBIngressControllerIAMPolicy IAM ポリシーをアタッチします。

In [23]:
!aws iam attach-role-policy \
--policy-arn {ALBPolicyArn} \
--role-name {workerRolearn}

次のコマンドを使用して、ALB Ingress Controller で使用するサービスアカウント、クラスターロール、およびクラスターロールバインディングを作成します。

In [24]:
!kubectl apply -f https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/rbac-role.yaml

clusterrole.rbac.authorization.k8s.io/alb-ingress-controller created
clusterrolebinding.rbac.authorization.k8s.io/alb-ingress-controller created
serviceaccount/alb-ingress-controller created


ALB Ingress Controller のマニフェストをダウンロード

In [132]:
!wget -O temp/alb-ingress-controller.yaml https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/alb-ingress-controller.yaml

--2020-01-27 02:50:00--  https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/alb-ingress-controller.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3129 (3.1K) [text/plain]
Saving to: ‘temp/alb-ingress-controller.yaml’

temp/alb-ingress-co 100%[===================>]   3.06K  --.-KB/s    in 0s      

2020-01-27 02:50:00 (42.6 MB/s) - ‘temp/alb-ingress-controller.yaml’ saved [3129/3129]



マニフェスト内容を確認

In [94]:
!cat temp/alb-ingress-controller.yaml

# Application Load Balancer (ALB) Ingress Controller Deployment Manifest.
# This manifest details sensible defaults for deploying an ALB Ingress Controller.
# GitHub: https://github.com/kubernetes-sigs/aws-alb-ingress-controller
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app.kubernetes.io/name: alb-ingress-controller
  name: alb-ingress-controller
  # Namespace the ALB Ingress Controller should run in. Does not impact which
  # namespaces it's able to resolve ingress resource for. For limiting ingress
  # namespace scope, see --watch-namespace.
  namespace: kube-system
spec:
  selector:
    matchLabels:
      app.kubernetes.io/name: alb-ingress-controller
  template:
    metadata:
      labels:
        app.kubernetes.io/name: alb-ingress-controller
    spec:
      containers:
        - name: alb-ingress-controller
          args:
            # Limit the namespace where this ALB Ingress Controller deployment will
            # resolve ingress resources. If left comment

-ingress-class=alb 行の後に、クラスター名、VPC ID、および AWS リージョン名を追加し、ファイルを保存して閉じます。



In [27]:
vpcID = publicSubnets['Subnets'][0]['VpcId']

In [95]:
import re

before_str='- --ingress-class=alb'
after_str='''
            - --ingress-class=alb
            - --cluster-name={clusterName}
            - --aws-vpc-id={vpcID}
            - --aws-region=us-west-2
'''.format(clusterName=clusterName,vpcID=vpcID).strip()

f = open('temp/alb-ingress-controller.yaml','r')
body = f.read()
new_body = re.sub(before_str, after_str, body, flags=re.DOTALL)
f.close()

f = open('temp/alb-ingress-controller.yaml','w')
f.write(new_body)
f.close()

In [96]:
!cat temp/alb-ingress-controller.yaml

# Application Load Balancer (ALB) Ingress Controller Deployment Manifest.
# This manifest details sensible defaults for deploying an ALB Ingress Controller.
# GitHub: https://github.com/kubernetes-sigs/aws-alb-ingress-controller
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app.kubernetes.io/name: alb-ingress-controller
  name: alb-ingress-controller
  # Namespace the ALB Ingress Controller should run in. Does not impact which
  # namespaces it's able to resolve ingress resource for. For limiting ingress
  # namespace scope, see --watch-namespace.
  namespace: kube-system
spec:
  selector:
    matchLabels:
      app.kubernetes.io/name: alb-ingress-controller
  template:
    metadata:
      labels:
        app.kubernetes.io/name: alb-ingress-controller
    spec:
      containers:
        - name: alb-ingress-controller
          args:
            # Limit the namespace where this ALB Ingress Controller deployment will
            # resolve ingress resources. If left comment

In [97]:
!kubectl apply -f temp/alb-ingress-controller.yaml

deployment.apps/alb-ingress-controller configured


# EFSの構築

■IAMロールの一覧取得

■NodeInstanceRoleのロール名をコピー（eksctl-aws-eks-0108-nodegroup-ng-NodeInstanceRole-XGXVUO8O861C）

In [31]:
row = !aws iam list-roles | grep '{clusterName}-nodegroup-n-NodeInstanceRole-'


import re

m = re.search(r'"RoleName": "(.*)",', row[0])
NodeInstanceRole = m.group(1)
print(NodeInstanceRole)

eksctl-aws-eks-200127-nodegroup-n-NodeInstanceRole-B5AZPMU1PGAA


■ロールにEFSのポリシーをアタッチ

In [32]:
!aws iam attach-role-policy --role-name \
{NodeInstanceRole} \
--policy-arn arn:aws:iam::aws:policy/AmazonElasticFileSystemFullAccess

■!awsを入れて、再実行→成功 (プレビュー版ではなく、正式版になっている)

In [33]:
!aws efs create-file-system \
--creation-token "EFS-topse" \
--performance-mode "generalPurpose"

{
    "OwnerId": "565991835156",
    "CreationToken": "EFS-topse",
    "FileSystemId": "fs-92bb0f38",
    "CreationTime": 1580090723.0,
    "LifeCycleState": "creating",
    "NumberOfMountTargets": 0,
    "SizeInBytes": {
        "Value": 0,
        "ValueInIA": 0,
        "ValueInStandard": 0
    },
    "PerformanceMode": "generalPurpose",
    "Encrypted": false,
    "ThroughputMode": "bursting",
    "Tags": []
}


■EFSの作成確認とFileSystemIdの取り出し

In [34]:
row = !aws efs describe-file-systems --creation-token "EFS-topse" | grep FileSystemId
print(row)

import re
m = re.search(r'"FileSystemId": "(.*)",', row[0])
FileSystemId = m.group(1)
print(FileSystemId)

['            "FileSystemId": "fs-92bb0f38",']
fs-92bb0f38


■Nameタグ付け

In [35]:
!aws efs create-tags --file-system-id {FileSystemId} --tags "Key=Name,Value={clusterName}"

■Nameタグ付け確認

In [36]:
!aws efs describe-tags --file-system-id {FileSystemId}

{
    "Tags": [
        {
            "Key": "Name",
            "Value": "aws-eks-200127"
        }
    ]
}


■eksのセキュリティグループとサブネットIDの確認

クラスターセキュリティーグループのIDの取得

In [37]:
row = !aws eks describe-cluster --name {clusterName}

import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"

clusterSecurityGroupId =json.loads(''.join(row))['cluster']['resourcesVpcConfig']['clusterSecurityGroupId']
pprint.pprint(clusterSecurityGroupId)


'sg-0c89e8bbad963aa10'


■マウントターゲット作成

プライベートのサブネットを指定する

In [38]:
!aws efs create-mount-target \
--file-system-id {FileSystemId} \
--subnet-id {privateSubnets['Subnets'][0]['SubnetId']} \
--security-group {clusterSecurityGroupId}

{
    "OwnerId": "565991835156",
    "MountTargetId": "fsmt-12c485ba",
    "FileSystemId": "fs-92bb0f38",
    "SubnetId": "subnet-0914edd00a3a9a9e5",
    "LifeCycleState": "creating",
    "IpAddress": "10.0.65.148",
    "NetworkInterfaceId": "eni-01dbe9ea57686e088"
}


In [39]:
!aws efs create-mount-target \
--file-system-id {FileSystemId} \
--subnet-id {privateSubnets['Subnets'][1]['SubnetId']} \
--security-group {clusterSecurityGroupId}

{
    "OwnerId": "565991835156",
    "MountTargetId": "fsmt-13c485bb",
    "FileSystemId": "fs-92bb0f38",
    "SubnetId": "subnet-07d10435b2f2773af",
    "LifeCycleState": "creating",
    "IpAddress": "10.0.114.195",
    "NetworkInterfaceId": "eni-039a5cc2931f9ed4b"
}


■マウントターゲット作成確認

In [40]:
!aws efs describe-mount-targets \
--file-system-id {FileSystemId}

{
    "MountTargets": [
        {
            "OwnerId": "565991835156",
            "MountTargetId": "fsmt-12c485ba",
            "FileSystemId": "fs-92bb0f38",
            "SubnetId": "subnet-0914edd00a3a9a9e5",
            "LifeCycleState": "creating",
            "IpAddress": "10.0.65.148",
            "NetworkInterfaceId": "eni-01dbe9ea57686e088"
        },
        {
            "OwnerId": "565991835156",
            "MountTargetId": "fsmt-13c485bb",
            "FileSystemId": "fs-92bb0f38",
            "SubnetId": "subnet-07d10435b2f2773af",
            "LifeCycleState": "creating",
            "IpAddress": "10.0.114.195",
            "NetworkInterfaceId": "eni-039a5cc2931f9ed4b"
        }
    ]
}


セキュリティグループの取り出し

In [41]:
row = !aws ec2 describe-security-groups
import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
SecurityGrrowoups =json.loads(''.join(row))

ControlPlaneSecurityGroup = ""
ClusterSharedNodeSecurityGroup = ""

for sg in SecurityGrrowoups['SecurityGroups']:
    
    if  "ControlPlaneSecurityGroup" in sg["GroupName"]:
        ControlPlaneSecurityGroup = sg["GroupId"]
    
    if "ClusterSharedNodeSecurityGroup" in sg["GroupName"]:
        ClusterSharedNodeSecurityGroup = sg["GroupId"]

print(ControlPlaneSecurityGroup)
print(ClusterSharedNodeSecurityGroup)

sg-0f4acf78034eab547
sg-0d128064e566b1e11


セキュリティグループ間のNFSインバウンドルールの追加

In [42]:
!aws ec2 authorize-security-group-ingress --group-id {ControlPlaneSecurityGroup} \
--protocol tcp --port 2049 --source-group {ClusterSharedNodeSecurityGroup}

In [43]:
!aws ec2 authorize-security-group-ingress --group-id {ClusterSharedNodeSecurityGroup} \
--protocol tcp --port 2049 --source-group {ControlPlaneSecurityGroup}

efs-provisioner を使用して、Kubernetes の永続ボリュームとして EFS ストレージをマウントすることにより、Amazon EFS を Amazon EKS ポッドで使用できるようにする。
https://aws.amazon.com/jp/premiumsupport/knowledge-center/eks-pods-efs/

1.    StorageClass および ConfigMap マニフェストファイルをダウンロードするには、次のコマンドを実行します。

In [257]:
!wget -O temp/class.yaml https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/class.yaml
!wget -O temp/configmap.yaml https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/configmap.yaml

--2020-01-27 03:21:13--  https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/class.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108 [text/plain]
Saving to: ‘temp/class.yaml’

temp/class.yaml     100%[===================>]     108  --.-KB/s    in 0s      

2020-01-27 03:21:13 (12.7 MB/s) - ‘temp/class.yaml’ saved [108/108]

--2020-01-27 03:21:14--  https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/configmap.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 

In [242]:
!cat temp/class.yaml

kind: StorageClass
apiVersion: storage.k8s.io/v1
metadata:
  name: aws-efs
provisioner: example.com/aws-efs


In [243]:
!cat temp/configmap.yaml

apiVersion: v1
kind: ConfigMap
metadata:
  name: efs-provisioner
data:
  file.system.id: fs-47a2c22e
  aws.region: us-west-2
  provisioner.name: example.com/aws-efs
  dns.name: ""


２．configmap.yaml ファイルで、file.system.id および aws.region を EFS に適した値に変更します。 


In [244]:
region="us-west-2"

In [258]:
import re

before_str=r'''file.system.id: \S*
  aws.region: \S*'''
after_str='''file.system.id: {FileSystemId}
  aws.region: {region}   
'''.format(FileSystemId=FileSystemId,region=region).strip()

f = open('temp/configmap.yaml','r')
body = f.read()
new_body = re.sub(before_str, after_str, body, flags=re.DOTALL)
f.close()

f = open('temp/configmap.yaml','w')
f.write(new_body)
f.close()

In [259]:
!cat temp/configmap.yaml

apiVersion: v1
kind: ConfigMap
metadata:
  name: efs-provisioner
data:
  file.system.id: fs-92bb0f38
  aws.region: us-west-2
  provisioner.name: example.com/aws-efs
  dns.name: ""


3.    StorageClass および ConfigMap をクラスターに適用するには、次のコマンドを実行します。

In [260]:
!kubectl apply -f temp/class.yaml
!kubectl apply -f temp/configmap.yaml

storageclass.storage.k8s.io/aws-efs unchanged
configmap/efs-provisioner configured


4.    Deployment および ClusterRole マニフェストファイルをダウンロードするには、次のコマンドを実行します。

In [261]:
!wget -O temp/deployment.yaml https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/deployment.yaml
!wget -O temp/rbac.yaml https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/rbac.yaml

--2020-01-27 03:22:04--  https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/deployment.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1452 (1.4K) [text/plain]
Saving to: ‘temp/deployment.yaml’

temp/deployment.yam 100%[===================>]   1.42K  --.-KB/s    in 0s      

2020-01-27 03:22:04 (172 MB/s) - ‘temp/deployment.yaml’ saved [1452/1452]

--2020-01-27 03:22:05--  https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/rbac.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiti

In [265]:
!cat temp/rbac.yaml

kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: efs-provisioner-runner
rules:
  - apiGroups: [""]
    resources: ["persistentvolumes"]
    verbs: ["get", "list", "watch", "create", "delete"]
  - apiGroups: [""]
    resources: ["persistentvolumeclaims"]
    verbs: ["get", "list", "watch", "update"]
  - apiGroups: ["storage.k8s.io"]
    resources: ["storageclasses"]
    verbs: ["get", "list", "watch"]
  - apiGroups: [""]
    resources: ["events"]
    verbs: ["create", "update", "patch"]
---
kind: ClusterRoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: run-efs-provisioner
subjects:
  - kind: ServiceAccount
    name: efs-provisioner
     # replace with namespace where provisioner is deployed
    namespace: default
roleRef:
  kind: ClusterRole
  name: efs-provisioner-runner
  apiGroup: rbac.authorization.k8s.io
---
kind: Role
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: leader-locking-efs-provisioner
rules:
  - apiGroups: [""]

In [263]:
!cat temp/deployment.yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  name: efs-provisioner
---
kind: Deployment
apiVersion: apps/v1
metadata:
  name: efs-provisioner
spec:
  replicas: 1
  selector:
    matchLabels:
      app: efs-provisioner
  strategy:
    type: Recreate 
  template:
    metadata:
      labels:
        app: efs-provisioner
    spec:
      serviceAccount: efs-provisioner
      containers:
        - name: efs-provisioner
          image: quay.io/external_storage/efs-provisioner:latest
          env:
            - name: FILE_SYSTEM_ID
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: file.system.id
            - name: AWS_REGION
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: aws.region
            - name: DNS_NAME
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: dns.name
       

5.    deployment.yaml ファイルで、ボリューム設定を / の path に変更します。serverのドメインも変更します。

In [267]:
import re

before_str=r'''server: \S*
            path: \S*'''

after_str='''server: {FileSystemId}.efs.{region}.amazonaws.com
            path: /
'''.format(FileSystemId=FileSystemId,region=region).strip()

f = open('temp/deployment.yaml','r')
body = f.read()
new_body = re.sub(before_str, after_str, body, flags=re.DOTALL)
f.close()

f = open('temp/deployment.yaml','w')
f.write(new_body)
f.close()

In [269]:
!cat temp/deployment.yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  name: efs-provisioner
---
kind: Deployment
apiVersion: apps/v1
metadata:
  name: efs-provisioner
spec:
  replicas: 1
  selector:
    matchLabels:
      app: efs-provisioner
  strategy:
    type: Recreate 
  template:
    metadata:
      labels:
        app: efs-provisioner
    spec:
      serviceAccount: efs-provisioner
      containers:
        - name: efs-provisioner
          image: quay.io/external_storage/efs-provisioner:latest
          env:
            - name: FILE_SYSTEM_ID
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: file.system.id
            - name: AWS_REGION
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: aws.region
            - name: DNS_NAME
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: dns.name
       

6.    RBAC リソースと Deployment マニフェストを適用するには、次のコマンドを実行します。

In [270]:
!kubectl apply -f temp/rbac.yaml
!kubectl apply -f temp/deployment.yaml

clusterrole.rbac.authorization.k8s.io/efs-provisioner-runner unchanged
clusterrolebinding.rbac.authorization.k8s.io/run-efs-provisioner unchanged
role.rbac.authorization.k8s.io/leader-locking-efs-provisioner unchanged
rolebinding.rbac.authorization.k8s.io/leader-locking-efs-provisioner unchanged
serviceaccount/efs-provisioner unchanged
deployment.apps/efs-provisioner configured


7.永続ボリュームクレームをダウンロードするには、次のコマンドを実行します。

In [272]:
!wget -O temp/claim.yaml https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/claim.yaml

--2020-01-27 03:28:45--  https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/claim.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222 [text/plain]
Saving to: ‘temp/claim.yaml’

temp/claim.yaml     100%[===================>]     222  --.-KB/s    in 0s      

2020-01-27 03:28:45 (27.7 MB/s) - ‘temp/claim.yaml’ saved [222/222]



In [274]:
cat temp/claim.yaml

kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: efs
  annotations:
    volume.beta.kubernetes.io/storage-class: "aws-efs"
spec:
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 1Mi


In [275]:
!kubectl apply -f temp/claim.yaml

persistentvolumeclaim/efs unchanged


出来ていることを確認する

In [276]:
!kubectl get pod

NAME                               READY   STATUS    RESTARTS   AGE
efs-provisioner-5d54bbbd75-vbbm2   1/1     Running   0          32s


# RDS構築

プライベートなサブネットをサブネットグループとしてまとめる

In [57]:
SubnetGroupName = "mysqlsubnetgroup"
print(SubnetGroupName)

mysqlsubnetgroup


In [58]:
!aws rds create-db-subnet-group \
        --db-subnet-group-name "{SubnetGroupName}" \
        --db-subnet-group-description "mysql_db_subnet_group" \
        --subnet-ids "{privateSubnets['Subnets'][0]['SubnetId']}" "{privateSubnets['Subnets'][1]['SubnetId']}"

{
    "DBSubnetGroup": {
        "DBSubnetGroupName": "mysqlsubnetgroup",
        "DBSubnetGroupDescription": "mysql_db_subnet_group",
        "VpcId": "vpc-0416a0124d607a464",
        "SubnetGroupStatus": "Complete",
        "Subnets": [
            {
                "SubnetIdentifier": "subnet-07d10435b2f2773af",
                "SubnetAvailabilityZone": {
                    "Name": "us-west-2c"
                },
                "SubnetStatus": "Active"
            },
            {
                "SubnetIdentifier": "subnet-0914edd00a3a9a9e5",
                "SubnetAvailabilityZone": {
                    "Name": "us-west-2b"
                },
                "SubnetStatus": "Active"
            }
        ],
        "DBSubnetGroupArn": "arn:aws:rds:us-west-2:565991835156:subgrp:mysqlsubnetgroup"
    }
}


RDSの名前などを定義し、RDSを作成

In [5]:
db_name = "RDS-topse"
db_user = "root"
db_password = "password"

In [60]:
!aws rds create-db-instance \
    --db-instance-identifier "{db_name}" \
    --allocated-storage 5 \
    --db-instance-class db.t2.micro \
    --engine mysql \
    --master-username "{db_user}" \
    --master-user-password "{db_password}" \
    --db-subnet-group-name  "{SubnetGroupName}" \
    --vpc-security-group-ids "{ClusterSharedNodeSecurityGroup}" \
    --multi-az


{
    "DBInstance": {
        "DBInstanceIdentifier": "rds-topse",
        "DBInstanceClass": "db.t2.micro",
        "Engine": "mysql",
        "DBInstanceStatus": "creating",
        "MasterUsername": "root",
        "AllocatedStorage": 5,
        "PreferredBackupWindow": "11:04-11:34",
        "BackupRetentionPeriod": 1,
        "DBSecurityGroups": [],
        "VpcSecurityGroups": [
            {
                "VpcSecurityGroupId": "sg-0d128064e566b1e11",
                "Status": "active"
            }
        ],
        "DBParameterGroups": [
            {
                "DBParameterGroupName": "default.mysql5.7",
                "ParameterApplyStatus": "in-sync"
            }
        ],
        "DBSubnetGroup": {
            "DBSubnetGroupName": "mysqlsubnetgroup",
            "DBSubnetGroupDescription": "mysql_db_subnet_group",
            "VpcId": "vpc-0416a0124d607a464",
            "SubnetGroupStatus": "Complete",
            "Subnets": [
                {
                 

後で使うのでエンドポイントを取得しておく


In [10]:
row = !aws rds describe-db-instances \
    --db-instance-identifier "{db_name}"
import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
DBInstances =json.loads(''.join(row))

db_address = DBInstances["DBInstances"][0]["Endpoint"]["Address"]

print(db_address)

rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com


## データテーブル作成（必要性は要確認）

適当なポッドを立ち上げる

In [1]:
!kubectl apply -f nginx-deployment.yaml

error: the path "nginx-deployment.yaml" does not exist


RuntimeError: Unexpected exit code: 1

In [29]:
!kubectl get pod

NAME                         READY   STATUS    RESTARTS   AGE
nginx-6ccb948d4-96cc9        1/1     Running   0          9s
wordpress-685cb5c9b6-mqw5b   1/1     Running   0          36m


ターミナルを開き、以下のようにログインしてDBを作成する

```
kubectl exec -it nginx-6ccb948d4-96cc9  /bin/bash

apt update
apt install -y mariadb-client
mariadb -u root --password="password" -h user2-db.ciph33ijhh38.us-west-2.rds.amazonaws.com
CREATE DATABASE wordpress;
exit
exit
```



いらないポッドを削除する

In [ ]:
!kubectl delete  -f nginx-deployment.yaml

# Kubectl デプロイ定義ファイルの編集(いらんかも)

まずは確認

In [2]:
!cat yaml/wordpress-deploy.yaml

apiVersion: apps/v1

kind: Deployment
metadata:
  name: wordpress
  labels:
    app: wordpress
spec:
  replicas: 2
  selector:
    matchLabels:
      app: wordpress
  template:
    metadata:
      labels:
        app: wordpress
    spec:
      containers:
        - image: wordpress:4.8-apache
          name: wordpress
          volumeMounts:
            - name: efs-pvc
              mountPath: "/var/www/html/wp-content/uploads"
          env:
          - name: WORDPRESS_DB_HOST
            value: rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com:3306
          - name: WORDPRESS_DB_PASSWORD
            value: password

          ports:
            - containerPort: 80
              name: wordpress
          lifecycle:
            postStart:
              exec:
                command: ["/bin/sh", "-c", "chown -R www-data:www-data /var/www/html/wp-content/uploads"]
      volumes:
        - name: efs-pvc
          persistentVolumeClaim:
            claimName: efs


RDSのアドレスを変更する

In [12]:
import re

before_str=r'''- name: WORDPRESS_DB_HOST
            value: \S*'''

after_str='''- name: WORDPRESS_DB_HOST
            value: {db_address}
'''.format(db_address=db_address).strip()

f = open('yaml/wordpress-deploy.yaml','r')
body = f.read()
new_body = re.sub(before_str, after_str, body, flags=re.DOTALL)
f.close()

f = open('yaml/wordpress-deploy.yaml','w')
f.write(new_body)
f.close()

In [13]:
!cat yaml/wordpress-deploy.yaml

apiVersion: apps/v1

kind: Deployment
metadata:
  name: wordpress
  labels:
    app: wordpress
spec:
  replicas: 2
  selector:
    matchLabels:
      app: wordpress
  template:
    metadata:
      labels:
        app: wordpress
    spec:
      containers:
        - image: wordpress:4.8-apache
          name: wordpress
          volumeMounts:
            - name: efs-pvc
              mountPath: "/var/www/html/wp-content/uploads"
          env:
          - name: WORDPRESS_DB_HOST
            value: rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com
          - name: WORDPRESS_DB_PASSWORD
            value: password

          ports:
            - containerPort: 80
              name: wordpress
          lifecycle:
            postStart:
              exec:
                command: ["/bin/sh", "-c", "chown -R www-data:www-data /var/www/html/wp-content/uploads"]
      volumes:
        - name: efs-pvc
          persistentVolumeClaim:
            claimName: efs
